**ILP**

In [ ]:
import time
from ortools.linear_solver import pywraplp
import networkx as nx

start_time = time.time()
G = nx.path_graph(20)

def main():
    n = len(list(G.nodes))
    solver = pywraplp.Solver.CreateSolver('SCIP')
    if not solver:
        return

    # variables
    f = [[solver.BoolVar(f'f[{i}][{j}]') for j in range(n)] for i in range(n)]
    b = [[solver.BoolVar(f'b[{i}][{j}]') for j in range(n)] for i in range(n)]
    c = [solver.BoolVar(f'c{i}') for i in range(n)]
    i = solver.IntVar(0, n, "i")
    v = solver.IntVar(0, n, "v")

    # Constraints

    for i in range(n):
        for j in range(n-1):
            solver.Add(b[i][j] >= f[i][j])

    for i in range(n):
        solver.Add(sum(f[i]) <= 1)

    for i in range(n-1):
        for j in range(n-1):
            solver.Add(b[i + 1][j] >= b[i][j])

    for j in range(n-1):
        solver.Add(b[0][j] == 0)

    for i in range(n):
        for j in range(n-1):
            solver.Add(c[i] <= b[i][j])

    for i in range(n):
        solver.Add((sum(b[i]) - n + 1) <= c[i])

    for i in range(n-1):
        for j in range(n-1):
            for k in G[j]:
                solver.Add(b[i + 1][k] >= b[i][j])

    for i in range(n-1):
        for j in range(n-1):
            sum_var = sum(b[i][k] for k in G[j])
            solver.Add(b[i + 1][j] <= sum_var + f[i + 1][j])

    objective = solver.Objective()
    for i in range(n):
        objective.SetCoefficient(c[i], 1)

    objective.SetMaximization()


    status = solver.Solve()


    if status == pywraplp.Solver.OPTIMAL:
        print('Burning Number =', n - solver.Objective().Value())




main()
end_time = time.time()
running_time=end_time - start_time
print('Running Time:', running_time, 'seconds')

Burning Number = 5.0
Running Time: 0.624652624130249 seconds


**Exact Algorithm**

In [ ]:
import networkx as nx
from itertools import chain, combinations
import matplotlib.pyplot as plt
import time

start_time = time.time()

G = nx.path_graph(20)


def neighbors(G):
    ng = {}
    for i in G.nodes():
        modng = list(G[i]) + [i]
        ng[i] = modng
    return ng


neighbors(G)
nbrs = neighbors(G)


def neighborsset(v):
    a = []
    for i in v:
        a += nbrs[i]
    return set(a)


def powerset(iterable):
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s) + 1))


def subsets(G):
    i = []
    for node in powerset(G.nodes):
        i.append(tuple(node))
    return i


def Diff(li1, li2):
    li_dif = [i for i in li1 + li2 if i not in li1 or i not in li2]
    return li_dif


cg = nx.Graph()


def configurationGraph(v):
    cg.add_nodes_from(v)

    edges_to_add = []  # List to store edges

    for i in cg.nodes:
        lis = list()
        if i != ():
            for j in i:
                lis += nbrs[j]
        lis = list(set(lis))
        rem_ver = list()
        rem_ver = Diff(list(G.nodes), list(i))
        for j in rem_ver:
            lis1 = lis + [j]
            edges_to_add.append((i, tuple(set(lis1))))  # Add edges to the list

    cg.add_edges_from(edges_to_add)  # Add edges outside the loop
    print("Burning Sequence :",nx.shortest_path(cg,source=(),target=v[-1]))
    print("Burning Number :",len(nx.shortest_path(cg,source=(),target=v[-1]))-1)
    #pos = nx.spring_layout(cg)
    #nx.draw(cg, pos)
    #labels = {node: node for node in cg.nodes}
    #nx.draw_networkx_labels(cg, pos, labels)

    #plt.show()

configurationGraph(subsets(G))

end_time = time.time()

print("Running time :",end_time-start_time)

Burning Sequence : [(), (0,), (0, 1, 7), (0, 1, 2, 6, 7, 8, 13), (0, 1, 2, 3, 5, 6, 7, 8, 9, 12, 13, 14, 17), (0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19)]
Burning Number : 5
Running time : 42.73422622680664


**Brute force**

In [ ]:
import time
def check(burned):
    for i in burned:
        if not i:
            return False
    return True

def burn(v, burned):
    burned[v] = 1


def nextpermutation(a):
    # Find the largest index i such that a[i] < a[i + 1]. If no such
    # index exists, the permutation is the last permutation
    for i in reversed(range(len(a) - 1)):
        if a[i] < a[i + 1]:
            break  # found
    else:  # no break: not found
        return False  # no next permutation

    # Find the largest index j greater than i such that a[i] < a[j]
    j = next(j for j in reversed(range(i + 1, len(a))) if a[i] < a[j])

    # Swap the value of a[i] with that of a[j]
    a[i], a[j] = a[j], a[i]

    # Reverse sequence from a[i + 1] up to and including the final element a[n]
    a[i + 1:] = reversed(a[i + 1:])
    return True

def propagate(burned, g, v):
    for i in range(v):
        if burned[i] == 1:
            size = len(g[i])
            for j in range(size):
                if burned[g[i][j]] == 0:
                    burned[g[i][j]] = 2
    for i in range(v):
        if burned[i] == 2:
            burned[i] = 1

def rounds(seq, burned, g, v):
    for i in range(v):
        propagate(burned, g, v)
        burn(seq[i], burned)
        if check(burned):
            return i + 1
    return v

v, e = map(int, input("Enter the number of vertices and edges in the graph: ").split())

g = [[] for _ in range(v)]

if e:
    print("Enter the edges:")

for _ in range(e):
    edge = input().split()
    a, b = map(int, edge)
    g[a].append(b)
    g[b].append(a)

start_time = time.time()
seq = list(range(v))
b = v

while True:
    burned = [0] * v  # Reset the burned list before each call to rounds
    b = min(b, rounds(seq, burned, g, v))
    if not nextpermutation(seq):
        break

print("The burning number is:",b)

end_time = time.time()
print("Running time :",end_time-start_time)

Enter the number of vertices and edges in the graph: 20 19
Enter the edges:
0 1
1 2
2 3
3 4
4 5
5 6
6 7
7 8
8 9
9 10
10 11
11 12
12 13
13 14
14 15
15 16
16 17
17 18
18 19


KeyboardInterrupt: ignored